In [8]:
# Variables always start in lower-case, upper-case is reserverd for class names such as SedGen here.
# This won't break anything, but it's a consensus used in the Python community.
def runmodel_getcriticalvolume(minerals, modal_mineralogy, csds_means, csds_stds, deviation):
    # No need for this parameter here as you overwrite in the else statement anyway
#     par_rock_old = 1e9
    par_rock_new = 1e9  
    lower_bound =  modal_mineralogy-(deviation*modal_mineralogy)
    upper_bound =  modal_mineralogy+(deviation*modal_mineralogy)
    counter = 0
    while counter != 50: 
        # Remove the discretization_init parameter temporarily to avoid
        # an error. It will be included in the next update of SedGen.
        model = SedGen(minerals, par_rock_new, modal_mineralogy, csds_means, csds_stds, learning_rate=10000, discretization_init=False)
        proposition1 = model.simulated_volume/par_rock_new >= lower_bound
        proposition2 = model.simulated_volume/par_rock_new <= upper_bound
        if np.all(proposition1 == True) and np.all(proposition2 == True):
            # You can use 'f-strings' to more easily format your strings and include variables in them.
            # You designate an f-string by placing an f before the single or double quotation marks.
            print (f"De modale mineralogy komt overeen voor een parent rock volume van: {par_rock_new} vierkante mm.")
            par_rock_new /= 5
#             par_rock_old /= 5
            counter += 1
        else: 
            print(f"De modale mineralogy komt niet overeen voor een parent rock volume van: {par_rock_new} vierkante mm.")
            par_rock_old = par_rock_new * 5
            par_rock_new = (par_rock_old + par_rock_new)/2
            counter += 1
    print(par_rock_old)
    return par_rock_old

In [9]:
import numpy as np
import numba as nb
from numba.typed import List
import pandas as pd
import itertools
from operator import itemgetter
import gc

from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof

In [10]:
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing

In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


____

In [13]:
minerals = ["Q", "P", "K", "B", "O", "A"]
modal_mineralogy = np.array([0.30591989, 0.38159713, 0.26209888, 0.0188256 , 0.00799247, 0.02356603])
csds_CA_NS_means = np.array([0.309, 0.330, 0.244, 0.223, 0.120, 0.122])
csds_CA_NS_stds = np.array([0.823, 0.683, 0.817, 0.819, 0.554, 0.782])

In [14]:
runmodel_getcriticalvolume(minerals, modal_mineralogy, csds_CA_NS_means, csds_CA_NS_stds, deviation=0.05)

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 1 3 ... 0 0 0]

Counting interfaces...
Correcting interface arrays for consistency...
too much Q 1
all good P 0
all good K 0
all good B 0
all good O 0
too few A -1
Initializing crystal size array... |Q|P|K|B|O|A|
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...

---SedGen model initialization finished succesfully---
De modale mineralogy komt overeen voor een parent rock volume van: 1000000000.0 vierkante mm.
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]

Counting interfaces...
Correcting interface arrays for consistency...
to

79960185.08417732

In [8]:
%debug

> d:\box sync\programming\python\jupyter_notebooks\sedgen\sedgen\src\sedgen\creation.py(224)perform_interface_array_correction()
    222                 # Add newly formed interfaces to interface_frequencies_corr
    223                 pair_index = (interface_array_corr[-1], index)
--> 224                 interface_frequencies_corr[pair_index] += prob_unit
    225                 # Add crystals to interface_array_corr
    226                 interface_array_corr = \



ipdb>  self.transitions_per_mineral[0][140761]


*** IndexError: index 140761 is out of bounds for axis 0 with size 140761


ipdb>  self.transitions_per_mineral[0][140760]


0


...

ipdb>  self.transitions_per_mineral.shape


*** AttributeError: 'tuple' object has no attribute 'shape'


ipdb>  self.transitions_per_mineral[0].shape


(140761,)


ipdb>  quit()


In [16]:
%debug

> d:\box sync\programming\python\jupyter_notebooks\sedgen\sedgen\src\sedgen\creation.py(224)perform_interface_array_correction()
    222                 # Add newly formed interfaces to interface_frequencies_corr
    223                 pair_index = (interface_array_corr[-1], index)
--> 224                 interface_frequencies_corr[pair_index] += prob_unit
    225                 # Add crystals to interface_array_corr
    226                 interface_array_corr = \



ipdb>  creation.create_interface_array(self.minerals_N, self.transitions_per_mineral)


*** NameError: name 'creation' is not defined


ipdb>  from sedgen import creation
ipdb>  creation.create_interface_array(self.minerals_N, self.transitions_per_mineral)


[0 0 0 ... 0 0 0]
[ 1  1  1 ...  0  0 33]
array([ 1,  1,  1, ...,  0,  0, 33], dtype=uint8)


...

ipdb>  int(np.sum(self.minerals_N))


629322


ipdb>  self.interface_array.shape


(629322,)


ipdb>  quit()


In [18]:
np.array([0] * 6, dtype=np.uint32)
np.zeros(6, dtype=np.uint32)

array([0, 0, 0, 0, 0, 0], dtype=uint32)

array([0, 0, 0, 0, 0, 0], dtype=uint32)

In [14]:
%debug

> d:\box sync\programming\python\jupyter_notebooks\sedgen\sedgen\src\sedgen\creation.py(224)perform_interface_array_correction()
    222                 # Add newly formed interfaces to interface_frequencies_corr
    223                 pair_index = (interface_array_corr[-1], index)
--> 224                 interface_frequencies_corr[pair_index] += prob_unit
    225                 # Add crystals to interface_array_corr
    226                 interface_array_corr = \



ipdb>  pair_index


(25, 1)


ipdb>  self.interface_array


array([ 1,  1,  1, ...,  0,  0, 25], dtype=uint8)


...

ipdb>  from sedgen import creation
ipdb>  creation.create_interface_array(self.minerals_N, self.transitions_per_mineral)


array([ 1,  1,  1, ...,  0,  0, 25], dtype=uint8)


ipdb>  creation.create_interface_array(self.minerals_N, self.transitions_per_mineral)


array([ 1,  1,  1, ...,  0,  0, 25], dtype=uint8)


ipdb>  quit()


In [ ]:
sedge